In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import date_format,year,col,to_timestamp

In [0]:
amazon_df = spark.read.csv("dbfs:/FileStore/tables/Amazon_all_transaction_1.csv",header = True, inferSchema = True)

amazon_df.printSchema()

root
 |-- ProductCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- SubCategory: string (nullable = true)
 |-- ActualPrice: double (nullable = true)
 |-- Marginpercentage: integer (nullable = true)
 |-- Discountpercentage: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- TransactionAmount: double (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Mode: string (nullable = true)
 |-- InvoiceNumber: integer (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- Country: string (nullable = true)



#a.	Find the top five performing mobile brands for the months of November, December and January r each year.

In [0]:
amazon_df1 = amazon_df.withColumn("Year",year(amazon_df.InvoiceDate)).withColumn("Month",month(amazon_df.InvoiceDate)).withColumn('quarter',quarter(amazon_df.InvoiceDate))
amazon_df1.show()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|Year|Month|quarter|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|2015|    1| 

In [0]:
amazon_df1.createOrReplaceTempView("Data_Table")

In [0]:
spark.sql("select Brand, Month, Year, sum(Quantity) as Total_sold from Data_Table where SubCategory='mobile' and Month in (11,12,1) group by Year, Month, Brand order by Total_sold DESC limit 5").show()

+-------+-----+----+----------+
|  Brand|Month|Year|Total_sold|
+-------+-----+----+----------+
|samsung|   12|2015|       517|
|samsung|   11|2015|       448|
|samsung|    1|2018|       434|
|samsung|   11|2016|       410|
|samsung|    1|2017|       395|
+-------+-----+----+----------+



#b.	Find the top five selling products for the months of November, December and January for each year.

In [0]:
spark.sql("select Brand,SubCategory,sum(Quantity) as Total_sold from Data_Table where Month in (11,12,1) group by Brand, SubCategory order by Total_sold DESC limit 5").show()

+-------+-----------+----------+
|  Brand|SubCategory|Total_sold|
+-------+-----------+----------+
|samsung|     mobile|      3974|
|   nike|men's shoes|      2695|
|    blu|     mobile|      1892|
|  apple|     mobile|      1818|
|  nokia|     mobile|      1467|
+-------+-----------+----------+



#a.	Number of products sold along with revenue aggregated quarterly

In [0]:
spark.sql("select Year, quarter,sum(Quantity) as Num_Product_Sold,sum(TransactionAmount) as Total_Revenue from Data_Table group by Year, quarter order by Year,quarter").show()

+----+-------+----------------+-------------------+
|Year|quarter|Num_Product_Sold|      Total_Revenue|
+----+-------+----------------+-------------------+
|2015|      1|           42693|       3.08548555E8|
|2015|      2|           42069|2.23020302445285E14|
|2015|      3|           43224|       3.17578576E8|
|2015|      4|           45636|       3.22182882E8|
|2016|      1|           49125|2.23020349928595E14|
|2016|      2|           41872|4.46040332280323E14|
|2016|      3|           44353|6.69060331721781E14|
|2016|      4|           42684|4.46040321418626E14|
|2017|      1|           44917|       3.23175489E8|
|2017|      2|           44283|       3.45020711E8|
|2017|      3|           40429|       2.91143349E8|
|2017|      4|           45000|4.46040344731496E14|
|2018|      1|           42619|6.69060301392097E14|
|2018|      2|            3905|        3.0780533E7|
+----+-------+----------------+-------------------+



#b.	Number of products sold in each category aggregated quarterly

In [0]:
spark.sql("select SubCategory,year,quarter,sum(Quantity) as Num_Product_Sold from Data_Table group by year,SubCategory,quarter order by Num_Product_Sold desc").show()

+-----------------+----+-------+----------------+
|      SubCategory|year|quarter|Num_Product_Sold|
+-----------------+----+-------+----------------+
|           mobile|2015|      4|            8374|
|all women's shoes|2016|      1|            8248|
|all women's shoes|2017|      4|            7947|
|all women's shoes|2016|      3|            7689|
|           mobile|2017|      2|            7423|
|all women's shoes|2017|      1|            7368|
|all women's shoes|2015|      3|            7278|
|all women's shoes|2015|      4|            7190|
|all women's shoes|2015|      1|            7155|
|all women's shoes|2016|      2|            7125|
|      men's shoes|2016|      1|            6921|
|all women's shoes|2015|      2|            6908|
|           mobile|2016|      3|            6869|
|all women's shoes|2017|      2|            6833|
|           mobile|2016|      4|            6810|
|           mobile|2017|      1|            6767|
|           mobile|2016|      1|            6733|


#c.	Number of products sold for each brand aggregated quarterly.

In [0]:
spark.sql("select Brand,year,quarter,sum(Quantity) as Num_Product_Sold from Data_Table group by year,Brand,quarter order by Num_Product_Sold desc").show()

+-------+----+-------+----------------+
|  Brand|year|quarter|Num_Product_Sold|
+-------+----+-------+----------------+
|   nike|2016|      1|            2902|
|   nike|2017|      1|            2743|
|   nike|2017|      2|            2731|
|   nike|2016|      4|            2729|
|   nike|2017|      4|            2645|
|   nike|2015|      1|            2642|
|   nike|2018|      1|            2623|
|   nike|2016|      3|            2544|
|   nike|2015|      3|            2520|
|   nike|2015|      4|            2520|
|   nike|2016|      2|            2399|
|   nike|2015|      2|            2336|
|   nike|2017|      3|            2330|
|samsung|2015|      4|            1536|
|samsung|2017|      2|            1463|
|samsung|2016|      3|            1434|
|samsung|2017|      3|            1349|
|samsung|2016|      4|            1334|
|samsung|2017|      1|            1302|
|samsung|2016|      1|            1237|
+-------+----+-------+----------------+
only showing top 20 rows



#3a.	Top 5 profit margin generating products for a given month.

In [0]:
spark.sql("select Brand,SubCategory,(Marginpercentage-Discountpercentage) as profit,Month,year from Data_Table group by year,SubCategory,Brand,Month,(Marginpercentage-Discountpercentage) order by profit desc").show()

+----------+--------------------+------+-----+----+
|     Brand|         SubCategory|profit|Month|year|
+----------+--------------------+------+-----+----+
|    wrteam|              mobile|    20|    5|2015|
|       muk|women's athletic ...|    20|    3|2015|
|mysticdrop|women's casual shoes|    20|    5|2015|
|      mapa|   men's dress shoes|    20|    1|2015|
|      marc|         men's shoes|    20|    5|2015|
|   journee|     shoes & jewelry|    20|    1|2015|
|  loeffler|   all women's shoes|    20|    6|2015|
| unbranded|              sports|    20|    1|2015|
|   samsung|          subwoofers|    20|    6|2015|
| celebrity|   all women's shoes|    20|    3|2015|
|    indigi|              mobile|    20|    1|2015|
|       new|       wrist watches|    20|    4|2015|
|     crocs|     shoes & jewelry|    20|    1|2015|
|      plum|              mobile|    20|    2|2015|
|     micro|      sporting goods|    20|    3|2015|
|    wilson|               jeans|    20|    3|2015|
|      frye|

In [0]:
spark.sql("select Brand,SubCategory,Marginpercentage,Month,Year from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Marginpercentage desc ) as rn" + " from Data_Table) tmp where rn <= 5" ).show()

+---------+-----------+----------------+-----+----+
|    Brand|SubCategory|Marginpercentage|Month|Year|
+---------+-----------+----------------+-----+----+
|    bella|accessories|              30|    3|2015|
|  samsung|accessories|              30|    3|2015|
|    bella|accessories|              30|    3|2015|
|  samsung|accessories|              30|    3|2016|
|    black|accessories|              30|    3|2016|
| unlisted|accessories|              30|    4|2015|
|     drew|accessories|              20|    4|2015|
|blackhawk|accessories|              20|    4|2017|
|     drew|accessories|              20|    4|2017|
|  crystal|accessories|              10|    4|2015|
|    bella|accessories|              30|    6|2015|
|   unique|accessories|              30|    6|2016|
|   unique|accessories|              30|    6|2016|
| unlisted|accessories|              30|    6|2016|
|   unique|accessories|              30|    6|2017|
|  samsung|accessories|              30|    9|2016|
|  samsung|a

d
 # 4a.Brand wise segregation of best performing categories profits wise

# 4a.Brand wise segregation of best performing categories profits wise

In [0]:
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from Data_Table group by Brand,Category,(Marginpercentage-Discountpercentage) order by profit desc").show()

+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|               peter|   clothing|    20|
|             kenwood|electronics|    20|
|            castelli|   clothing|    20|
|                 dvs|   clothing|    20|
|               annie|   clothing|    20|
|              saddle|   clothing|    20|
|          underwraps|   clothing|    20|
|           haflinger|   clothing|    20|
|                 jbu|   clothing|    20|
|            muckboot|   clothing|    20|
|            siriusxm|electronics|    20|
|            funtasma|   clothing|    20|
|               perry|   clothing|    20|
|               donna|      shoes|    20|
|              noriko|   clothing|    20|
|                tumi|electronics|    20|
|                 cet|   clothing|    20|
|               nicki|      shoes|    20|
|yu&yu soccer men'...|   clothing|    20|
|               saint|   clothing|    20|
+--------------------+-----------+

#4b.	Brand wise segregation of least performing categories orders wise

In [0]:
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from Data_Table group by Brand,Category,(Marginpercentage-Discountpercentage) order by profit ").show()

+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|           alexander|   clothing|   -50|
|               ridge|   clothing|   -50|
|               bgood|   clothing|   -50|
|             pantech|electronics|   -50|
|            alliance|   clothing|   -50|
|             bellini|   clothing|   -50|
|             natural|   clothing|   -50|
|         ellie shoes|   clothing|   -50|
|                five|   clothing|   -50|
|fruit of the loom...|   clothing|   -50|
|                easy|   clothing|   -50|
|              report|   clothing|   -50|
|                lowa|   clothing|   -50|
|                 nwt|   clothing|   -50|
|             emporio|   clothing|   -50|
|            siriusxm|electronics|   -50|
|            onitsuka|   clothing|   -50|
|             dickies|   clothing|   -50|
|               bally|   clothing|   -50|
|                moda|   clothing|   -50|
+--------------------+-----------+

#5a.	Find the top five selling products in a given month

In [0]:
spark.sql("select SubCategory,Month,Year,sum(Quantity) as Total_sold from Data_Table group by SubCategory,Month,Year order by Total_sold desc").show()

+-----------------+-----+----+----------+
|      SubCategory|Month|Year|Total_sold|
+-----------------+-----+----+----------+
|all women's shoes|    2|2016|      3345|
|all women's shoes|   11|2017|      3260|
|           mobile|    3|2016|      3030|
|all women's shoes|    5|2017|      3015|
|all women's shoes|   12|2015|      2970|
|           mobile|   12|2015|      2935|
|all women's shoes|    8|2016|      2925|
|all women's shoes|    1|2015|      2891|
|all women's shoes|    3|2018|      2846|
|           mobile|    8|2017|      2787|
|           mobile|   10|2015|      2738|
|           mobile|    8|2016|      2727|
|           mobile|   11|2015|      2701|
|all women's shoes|    3|2016|      2660|
|      men's shoes|   11|2017|      2659|
|all women's shoes|    8|2017|      2646|
|all women's shoes|    2|2017|      2600|
|all women's shoes|   12|2017|      2596|
|all women's shoes|    5|2015|      2593|
|all women's shoes|    9|2015|      2587|
+-----------------+-----+----+----

In [0]:
spark.sql("select SubCategory,Month,sum(Quantity) as Total_sold from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Total_sold desc ) as rn" + " from Data_Table) tmp where rn <= 5" ).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-3604470008547335> in <module>
----> 1 spark.sql("select SubCategory,Month,sum(Quantity) as Total_sold from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Total_sold desc ) as rn" + " from Data_Table) tmp where rn <= 5" ).show()

/databricks/spark/python/pyspark/sql/session.py in sql(self, sqlQuery)
    775         [Row(f1=1, f2='row1'), Row(f1=2, f2='row2'), Row(f1=3, f2='row3')]
    776         """
--> 777         return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
    778 
    779     def table(self, tableName):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.ta

#5b.	Find the top five products having the highest margin in a given month

In [0]:
spark.sql("select Brand,Category,SubCategory,Month,Year,Marginpercentage from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month,Year Order by Marginpercentage desc ) as rn" + " from Data_Table) tmp where rn <= 5" ).show()

+-----------------+-----------+-----------+-----+----+----------------+
|            Brand|   Category|SubCategory|Month|Year|Marginpercentage|
+-----------------+-----------+-----------+-----+----+----------------+
|         unlisted|   clothing|accessories|    1|2018|              30|
|            bella|   clothing|accessories|    1|2018|              30|
|        bandolino|   clothing|accessories|    1|2018|              20|
|             drew|   clothing|accessories|    1|2018|              20|
|             dell|electronics|accessories|    1|2018|              10|
|          samsung|electronics|accessories|    2|2018|              30|
|        bandolino|   clothing|accessories|    2|2018|              20|
|        panasonic|electronics|accessories|    2|2018|              10|
|          samsung|electronics|accessories|    3|2016|              30|
|            black|   clothing|accessories|    3|2016|              30|
|          samsung|electronics|accessories|    3|2016|          

#5c.	Find the five lowest selling products in a given month.

In [0]:
spark.sql("select SubCategory,sum(Quantity) as Total,Month,Year from Data_Table group by SubCategory,Marginpercentage,Month,Year order by Total ").show()

+--------------------+-----+-----+----+
|         SubCategory|Total|Month|Year|
+--------------------+-----+-----+----+
|      heeled sandals|    1|    3|2015|
|       women's jeans|    1|    4|2015|
|     knee-high boots|    1|    2|2015|
|             apparel|    1|    6|2015|
|         golf gloves|    1|    3|2015|
|     womens costumes|    1|    3|2015|
|      sporting goods|    1|    2|2015|
|         photography|    1|    5|2015|
|         men's jeans|    1|    3|2015|
|see more mesh hal...|    1|    2|2015|
|  bags & accessories|    1|    4|2015|
|        men's shorts|    1|    2|2015|
|                  tv|    1|    4|2015|
|            slippers|    1|    3|2015|
|               women|    1|    5|2015|
|          headphones|    1|    1|2015|
|see more saxx 969...|    1|    2|2015|
|            slippers|    1|    4|2015|
|safety equipment ...|    1|    5|2015|
|     protective gear|    1|    1|2015|
+--------------------+-----+-----+----+
only showing top 20 rows



#6.	Trending brands for given month order wise

In [0]:
spark.sql("select Brand,SubCategory,Month,sum(Quantity) as Quantity_num from Data_Table group by Brand,Month,Subcategory order by Month,Quantity_num desc").show()

+----------+-----------------+-----+------------+
|     Brand|      SubCategory|Month|Quantity_num|
+----------+-----------------+-----+------------+
|   samsung|           mobile|    1|        1567|
|      nike|      men's shoes|    1|         941|
|     apple|           mobile|    1|         772|
|       blu|           mobile|    1|         680|
|     nokia|           mobile|    1|         589|
|        lg|           mobile|    1|         497|
|       htc|           mobile|    1|         472|
|  motorola|           mobile|    1|         390|
|      sony|           mobile|    1|         385|
|blackberry|           mobile|    1|         336|
|      nike|all women's shoes|    1|         319|
|      puma|      men's shoes|    1|         274|
|      nike|  shoes & jewelry|    1|         254|
|      nike|  all men's shoes|    1|         249|
|      vans|      men's shoes|    1|         223|
|     yu&yu|all women's shoes|    1|         217|
|       new|      men's shoes|    1|         211|
